In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time
import sys

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Установка seed для воспроизводимости
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# ==================== 1. НАСТРОЙКИ ====================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Устройство: {device} | PyTorch: {torch.__version__}\n")


In [ ]:
# ==================== 2. ЗАГРУЗКА ДАННЫХ ====================
print("Загрузка MNIST...")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

print(f"Тренировка: {len(train_dataset):,} изображений")
print(f"Тест: {len(test_dataset):,} изображений\n")

In [ ]:
# ==================== 3. АРХИТЕКТУРА CNN ====================
class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Свёрточная часть (сохраняет пространственную структуру!)
        self.conv_layers = nn.Sequential(
            # Слой 1: [B, 1, 28, 28] → [B, 32, 28, 28]
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),  # Стабилизирует обучение

            # Слой 2: [B, 32, 28, 28] → [B, 32, 14, 14]
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Уменьшаем размер в 2 раза
            nn.Dropout2d(0.25),  # Dropout для свёрточных карт

            # Слой 3: [B, 32, 14, 14] → [B, 64, 14, 14]
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),

            # Слой 4: [B, 64, 14, 14] → [B, 64, 7, 7]
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
        )

        # Классификатор (здесь нужен Flatten!)
        self.classifier = nn.Sequential(
            nn.Flatten(),  # [B, 64, 7, 7] → [B, 3136]
            nn.Linear(64 * 7 * 7, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)  # 10 классов цифр
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.classifier(x)
        return x

model = MNIST_CNN().to(device)

# Подсчёт параметров
total_params = sum(p.numel() for p in model.parameters())
conv_params = sum(p.numel() for name, p in model.named_parameters() if 'conv' in name.lower())
fc_params = sum(p.numel() for name, p in model.named_parameters() if 'fc' in name.lower() or 'classifier' in name.lower())

print("Архитектура CNN:")
print(model)
print(f"\n Параметры: Всего {total_params:,} | Свёрточные: {conv_params:,} | Полносвязные: {fc_params:,}\n")

In [ ]:
# ==================== 4. ОБУЧЕНИЕ ====================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

def evaluate(model, loader, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)

            total_loss += loss.item() * data.size(0)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += data.size(0)

    return total_loss / total, 100. * correct / total

# История обучения
train_losses, train_accs = [], []
test_losses = []
test_accs = []
learning_rates = []
best_acc = 0.0

epochs = 15
print("="*80)
print(" НАЧАЛО ОБУЧЕНИЯ CNN")
print("="*80)

In [1]:
start_time = time.time()

for epoch in range(1, epochs + 1):
    epoch_start = time.time()

    # Тренировка
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    current_lr = optimizer.param_groups[0]['lr']

    progress_bar = tqdm(train_loader, desc=f"Эпоха {epoch}/{epochs}", leave=False, file=sys.stdout)
    for data, target in progress_bar:
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        # Статистика
        running_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += data.size(0)

        # Прогресс-бар
        batch_acc = 100. * pred.eq(target).sum().item() / data.size(0)
        progress_bar.set_postfix({'loss': f"{loss.item():.4f}", 'acc': f"{batch_acc:.1f}%"})

    train_loss = running_loss / total
    train_acc = 100. * correct / total

    # Валидация
    test_loss, test_acc = evaluate(model, test_loader, device)

    # Scheduler
    prev_lr = optimizer.param_groups[0]['lr']
    scheduler.step(test_loss)
    new_lr = optimizer.param_groups[0]['lr']
    lr_changed = (new_lr != prev_lr)

    # Сохранение истории
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    learning_rates.append(new_lr)

    # Время эпохи
    epoch_time = time.time() - epoch_start

    # Вывод результатов
    print(f"\nЭпоха {epoch:2d}/{epochs} | Время: {epoch_time:.2f}с | LR: {current_lr:.2e}")
    print("-" * 80)
    print(f"  Тренировка: Лосс: {train_loss:.4f} | Точность: {train_acc:.2f}%")
    print(f"  Тест:       Лосс: {test_loss:.4f} | Точность: {test_acc:.2f}%")
    print(f"  ΔЛосс: {test_loss - train_loss:+.4f} | ΔТочность: {test_acc - train_acc:+.2f}%")

    if lr_changed:
        print(f"LR изменён: {prev_lr:.2e} → {new_lr:.2e}")

    # Сохранение лучшей модели
    if test_acc > best_acc:
        best_acc = test_acc
        best_epoch = epoch
        torch.save(model.state_dict(), 'best_cnn_model.pth')
        print(f"  Новая лучшая модель Точность: {best_acc:.2f}% (сохранено)")

    # Ранняя остановка
    if epoch > 8 and test_acc > 99.0:
        print(f"\n Достигнута отличная точность ({test_acc:.2f}%)! Завершаем обучение.")
        epochs_completed = epoch
        break
else:
    epochs_completed = epochs

total_time = time.time() - start_time
print("\n" + "="*80)
print(f"ОБУЧЕНИЕ ЗАВЕРШЕНО | Эпох: {epochs_completed} | Время: {total_time:.2f}с")
print("="*80)

# ==================== 5. ФИНАЛЬНАЯ ОЦЕНКА ====================
final_loss, final_acc = evaluate(model, test_loader, device)
print(f"\nФИНАЛЬНЫЙ РЕЗУЛЬТАТ НА ТЕСТЕ:")
print(f"   Лосс: {final_loss:.4f} | Точность: {final_acc:.2f}%")
print(f"   Лучшая точность: {best_acc:.2f}% (эпоха {best_epoch})")

# ==================== 6. ГРАФИКИ ====================
print("\nПостроение графиков.")

plt.style.use('seaborn-v0_8-darkgrid')
fig = plt.figure(figsize=(16, 5))

# Лосс
ax1 = plt.subplot(1, 3, 1)
epochs_range = range(1, len(train_losses) + 1)
ax1.plot(epochs_range, train_losses, 'o-', label='Train Loss', linewidth=2, markersize=6)
ax1.plot(epochs_range, test_losses, 's-', label='Test Loss', linewidth=2, markersize=6)
ax1.axvline(best_epoch, color='red', linestyle='--', alpha=0.7, label=f'Лучшая эпоха ({best_epoch})')
ax1.set_xlabel('Эпоха', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Функция потерь', fontsize=14, weight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Точность
ax2 = plt.subplot(1, 3, 2)
ax2.plot(epochs_range, train_accs, 'o-', label='Train Accuracy', linewidth=2, markersize=6)
ax2.plot(epochs_range, test_accs, 's-', label='Test Accuracy', linewidth=2, markersize=6)
ax2.axhline(99, color='green', linestyle=':', alpha=0.5, label='99% baseline')
ax2.axvline(best_epoch, color='red', linestyle='--', alpha=0.7)
ax2.set_xlabel('Эпоха', fontsize=12)
ax2.set_ylabel('Accuracy (%)', fontsize=12)
ax2.set_title('Точность', fontsize=14, weight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_ylim([95, 100])

# Learning Rate
ax3 = plt.subplot(1, 3, 3)
ax3.plot(epochs_range, learning_rates, '^-', color='purple', linewidth=2, markersize=6)
ax3.set_xlabel('Эпоха', fontsize=12)
ax3.set_ylabel('LR', fontsize=12)
ax3.set_title('Learning Rate', fontsize=14, weight='bold')
ax3.set_yscale('log')
ax3.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig('cnn_training_curves.png', dpi=150, bbox_inches='tight')
print("Графики сохранены в 'cnn_training_curves.png'")
plt.show()

# ==================== 7. ВИЗУАЛИЗАЦИЯ АКТИВАЦИЙ (ОПЦИОНАЛЬНО) ====================
print("\nВизуализация активаций свёрточных слоёв...")

model.eval()
with torch.no_grad():
    # Берём одно изображение
    sample_img, sample_label = test_dataset[0]
    sample_img = sample_img.unsqueeze(0).to(device)  # [1, 1, 28, 28]

    # Проход через первый свёрточный блок
    conv1 = nn.Sequential(*list(model.conv_layers.children())[:3])  # Первые 3 слоя: Conv+ReLU+BN
    activations = conv1(sample_img)

    # Визуализация
    fig = plt.figure(figsize=(15, 4))

    # Исходное изображение
    ax1 = plt.subplot(1, 4, 1)
    ax1.imshow(sample_img.cpu().squeeze(), cmap='gray')
    ax1.set_title(f'Исходное изображение\nЦифра: {sample_label}', fontsize=11)
    ax1.axis('off')

    # Активации (первые 8 карт признаков)
    for i in range(3):
        ax = plt.subplot(1, 4, i + 2)
        ax.imshow(activations[0, i].cpu(), cmap='viridis')
        ax.set_title(f'Карта признаков {i+1}', fontsize=11)
        ax.axis('off')

    plt.suptitle('Визуализация активаций первого свёрточного слоя', fontsize=14, weight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('cnn_activations.png', dpi=150, bbox_inches='tight')
    print("Активации сохранены в 'cnn_activations.png'")
    plt.show()

# ==================== 8. МАТРИЦА ОШИБОК ====================
print("\nПостроение матрицы ошибок...")

from sklearn.metrics import confusion_matrix
import seaborn as sns

all_preds = []
all_targets = []

model.eval()
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1)
        all_preds.extend(pred.cpu().numpy())
        all_targets.extend(target.cpu().numpy())

cm = confusion_matrix(all_targets, all_preds)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=range(10), yticklabels=range(10))
plt.xlabel('Предсказанный класс')
plt.ylabel('Истинный класс')
plt.title('Матрица ошибок CNN на MNIST', fontsize=14, weight='bold')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150, bbox_inches='tight')
print("Матрица ошибок сохранена в 'confusion_matrix.png'")
plt.show()

# ==================== 9. СОХРАНЕНИЕ МОДЕЛИ ====================
torch.save({
    'epoch': epochs_completed,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'test_acc': final_acc,
    'best_acc': best_acc,
    'best_epoch': best_epoch,
}, 'mnist_cnn_checkpoint.pth')

print(f"\nМодель сохранена в 'mnist_cnn_checkpoint.pth'")
print("лучшая модель сохранена в 'best_cnn_model.pth'")

print("\n" + "="*80)
print("CNN обучена. Точность на тесте: {:.2f}%".format(final_acc))
print("="*80)

NameError: name 'time' is not defined